### Run seed correlations on all subjects for an initial QA

the code is devided into two steps:
1. Regress out confounds.  (regressing out the confounds can be done by load_confounds instead of the method used here)
2. Calculate seed correlations and plot.

An outer for loop iterates over all subjects.

### Imports

In [11]:
from nilearn import image as nimg
from nilearn import plotting as nplot
from nilearn import masking
import pandas as pd
import numpy as np
from nilearn import input_data
import os

#### Prepare a list of all subjects for the loop. 
The names should match the subject dirctory name.


In [12]:
subs = ['Y01', 'Y02', 'Y03', 'Y04', 'Y05', 'Y06', 'Y07', 'Y08', 'Y10', 'Y11', 'Y12',
        'Y13', 'Y14','Y16', 'Y17', 'Y19', 'Y20']
descs = ['preproc', 'smoothAROMAnonaggr_bold']
runs = [1 ,2]

In [16]:
os.chdir('/Users/haya_/Desktop/fmriprep')
os.getcwd()


'C:\\Users\\haya_\\Desktop\\fmriprep'

#### Set other parameters for file selection.
The parametrs can be looped over by a for loop, but it is not possible to do so in a notebook, so I'll just use one subject.

In [46]:
run = 2
desc = 'smoothAROMAnonaggr_bold'
sub = 'Y01'

#### Load files:


In [47]:
func_file = f'sub-{sub}/ses-1/func/sub-{sub}_ses-1_task-rest_run-{run}_' \
                f'space-MNI152NLin6Asym_desc-{desc}.nii.gz'


confound_file = f'sub-{sub}/ses-1/func/sub-{sub}_ses-1_task-rest_run-{run}_' \
                    f'desc-confounds_timeseries.tsv'

confound_df = pd.read_csv(confound_file, delimiter='\t')

#### Load functinal image from the file path:

In [48]:
func_img = nimg.load_img(func_file)

#### Creating a brain mask for the functinal image. 
We do this because later we are going to be applying the mask to the img(when callling the clean_img function). And shape/dimensions of the img and the mask must match. This step is not needed if we are using a file that ends with "desc-preproc_bold.nii.gz", because fmriprep already supplies a matching mask for this file, but since we are not using the defult file I have to create a new matching mask.



In [49]:
brain_mask = masking.compute_brain_mask(func_img)

### Chossing confound regressors:

Original code from: https://github.com/carpentries-incubator/SDC-BIDS-fMRI/blob/128f3c3fbefd48c7e63ebd2660f45c0b93e31ec6/code/05_data-cleaning-with-nilearn_solutions.ipynb
                                                                  
The Yeo 2011 Pre-processing schema
    Confound regressors:
    6 motion parameters (trans_x, trans_y, trans_z, rot_x, rot_y, rot_z)
    Global signal (global_signal)
    Cerebral spinal fluid signal (csf)
    White matter signal (white_matter)

This is a total of 9 base confound regressor variables. 
then we add temporal derivatives of each of these signals as well (1 temporal derivative for each), the result is 18 confound regressors.
Temporal Derivatives are the changes in values across 2 consecutive samples. It represents change in signal over time.
For example, when dealing with the confound variable "X", which represents motion along the "trans_x" direction, 
the temporal derivative represents velocity in the X direction.

In [50]:
# here we pick our base confound regressors as done in Yeo 2011's pre-processing framework
confound_vars = ['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z', 'global_signal', 'csf',
                 'white_matter']

# Get derivative column names fMRIPrep provides this for us. Derivative columns are denoted as the following:
#     {NAME_OF_COLUMN}_derivative1

derivative_columns = ['{}_derivative1'.format(c) for c in confound_vars]

# print("derivative_columns:\n", derivative_columns)

final_confounds = confound_vars + derivative_columns

# print("final confounds\n", final_confounds)

confound_df = confound_df[final_confounds]

print("confound df head:\n", confound_df.head(10))

confound df head:
         trans_x   trans_y       trans_z     rot_x         rot_y     rot_z  \
0 -4.472990e-09  0.000000  3.333600e-09  0.000000 -7.801070e-05  0.000000   
1  1.928960e-05  0.088603  6.097940e-02 -0.000284 -9.464280e-05  0.000157   
2 -3.643880e-07 -0.045601  2.310660e-02  0.000039 -1.371490e-24  0.000202   
3  1.410900e-05  0.020789  4.190990e-02  0.000010 -1.532280e-04  0.000136   
4 -7.656770e-03  0.087604  3.237300e-02 -0.000347 -0.000000e+00  0.000179   
5 -1.129550e-02 -0.058754  3.237180e-02  0.000351 -2.785430e-04  0.000000   
6 -1.223040e-02  0.100739  6.111480e-02 -0.000006 -2.728880e-04  0.000000   
7 -9.972980e-03 -0.001088  2.236870e-02  0.000436 -1.953240e-04  0.000000   
8 -2.934020e-02  0.018361  3.247830e-02  0.000360 -6.942690e-06  0.000000   
9 -2.518650e-02  0.074818  5.013930e-02  0.000602  3.264110e-05  0.000000   

   global_signal         csf  white_matter  trans_x_derivative1  \
0     571.392746  487.973242    542.334672                  NaN   

#### Removing the first 4 timepoints from our functional image:

In [51]:
print("func_img shape\n", func_img.shape)

func_img = func_img.slicer[:, :, :, 4:]

print("func_img shape\n", func_img.shape)



func_img shape
 (91, 109, 91, 379)
func_img shape
 (91, 109, 91, 375)


#### Removing the first 4 timepoints from the confounds df

In [52]:
print("confound df's shape:\n", confound_df.shape)

confound_df = confound_df.loc[4:]

print("confound df's shape after droping first 4 trs:\n", confound_df.shape)  # number of rows should match that of the functional image
print("confound df head after droping first 4 trs:\n", confound_df.head(10))


confound df's shape:
 (379, 18)
confound df's shape after droping first 4 trs:
 (375, 18)
confound df head after droping first 4 trs:
      trans_x   trans_y   trans_z     rot_x     rot_y     rot_z  global_signal  \
4  -0.007657  0.087604  0.032373 -0.000347 -0.000000  0.000179     567.386158   
5  -0.011295 -0.058754  0.032372  0.000351 -0.000279  0.000000     568.906355   
6  -0.012230  0.100739  0.061115 -0.000006 -0.000273  0.000000     569.850669   
7  -0.009973 -0.001088  0.022369  0.000436 -0.000195  0.000000     569.966198   
8  -0.029340  0.018361  0.032478  0.000360 -0.000007  0.000000     569.812576   
9  -0.025187  0.074818  0.050139  0.000602  0.000033  0.000000     568.742639   
10 -0.035734 -0.021218  0.032483  0.001285  0.000013  0.000000     569.051718   
11 -0.032641  0.133985  0.076372  0.000852  0.000135  0.000072     568.509220   
12 -0.026049 -0.031052  0.025206  0.001222  0.000243  0.000000     568.539577   
13 -0.032168  0.084915  0.073131  0.001080  0.000099  0

#### Convert data frame into a matrix


In [58]:
confounds_matrix = confound_df.values

# Confirm matrix size is correct
print("confounds matrix shape:\n", confounds_matrix.shape)

confounds matrix shape:
 (375, 18)


#### Set some constants according to The Yeo 2011 Pre-processing schema


In [60]:
high_pass = 0.009  # filtering minimum
low_pass = 0.08  # filtering maximum
t_r = 1.4

### Clean! 

In [63]:
# mask file source shuold be the actual func file
clean_img = nimg.clean_img(func_img, confounds=confounds_matrix, detrend=True, standardize=True,
                               low_pass=low_pass, high_pass=high_pass, t_r=t_r, mask_img=brain_mask)

C:\Users\haya_\anaconda3\lib\site-packages\nilearn\image\image.py:1106: FutureWarning: The parameter "sessions" will be removed in 0.9.0 release of Nilearn. Please use the parameter "runs" instead.
  data = signal.clean(
